In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def thresh2binarygrid(img, gridsize=(10,10), percentage=0.20):
    """ Takes a thresholded image (i.e. from a cv2.inRange operation), divides
    the image into a grid of gridsize; if a gridblock has percentage white pixels,
    the gridblock will be "turned on"

    Parameters
    ----------
    img: np.array, shape (image pixel height, image pixel width)
        values should be between 0 - 255.

    gridsize: tuple, default (10, 10)

    percentage: float, between 0 - 1

    Returns
    -------
    grid: binary grid of size gridsize
    """
    gridrows, gridcols = gridsize
    m, n = img.shape
    
    # initialize binary grid
    grid = np.zeros(gridsize)
    
    # define the size of each grid block
    block_size = (m/gridrows, n/gridcols)

    for i in range(gridrows):
        for j in range(gridcols):
            block = img[int(block_size[0]*i):int(block_size[0]*(i+1)), int(block_size[1]*j):int(block_size[1]*(j+1))]

            # if percentage of the block was white (inRange)
            threshold = block_size[0] * block_size[1] * 255 * percentage
            if block.sum() >= threshold:
                # turn the gridblock on
                grid[i,j] = 1
    
    return grid

def get_bbox_from_grid(img, grid):
    """ Gets a bounding box in the form of (pt1, pt2), pairs of (x,y)
    coordinates the define the lefttop and rightbottom of the bounding rectangle """

    # get top
    for i in range(grid.shape[0]):
        if grid[i].sum() > 0:
            top = i - 1
            break
    
    # get left
    for j in range(grid.shape[1]):
        if grid[:, j].sum() > 0:
            left = j - 1
            break
        
    # get bottom
    for i in range(grid.shape[0] - 1, -1, -1):
        if grid[i].sum() > 0:
            bottom = i + 1
            break
            
    # get right
    for j in range(grid.shape[1] - 1, -1, -1):
        if grid[:, j].sum() > 0:
            right = j + 1
            break
    
    # (x, y)
    pt1 = (img.shape[1]/grid.shape[1]*left, img.shape[0]/grid.shape[0]*top)
    
    # (right+1) and (bottom+1) cuz grid only represents the left hand corner of a block
    pt2 = (img.shape[1]/grid.shape[1]*(right+1), img.shape[0]/grid.shape[0]*(bottom+1))
    
    return pt1, pt2

In [ ]:
MIN_MATCH_COUNT = 10
minyellow = np.array([23,175,130]) 
maxyellow = np.array([32,255,255])  

img2 = cv2.imread('images/leftturn_box_small.png',0) # #templateImage

img1_3channel = cv2.imread('images/uturn_box.png')
img1 = cv2.imread('images/leftturn_scene.jpg') # sceneImage

hsv_image = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
binarized_image = cv2.inRange(hsv_image, minyellow, maxyellow)
binaryGrid = thresh2binarygrid(binarized_image, gridsize=(15, 15), percentage=0.2)
pt1, pt2 = get_bbox_from_grid(binarized_image, binaryGrid)
cropped_sign = img1[pt1[1]:pt2[1], pt1[0]:pt2[0]]
img1 = cv2.cvtColor(cropped_sign, cv2.COLOR_BGR2GRAY)

In [ ]:
plt.imshow(img1)

In [ ]:
plt.imshow(img2)

In [ ]:
# Initiate SIFT detector
sift = cv2.SIFT()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

In [ ]:
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)

In [ ]:
# bf = cv2.BFMatcher()
# # matches = bf.knnMatch(des1, des2, k=2)

In [ ]:
# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)

In [ ]:
print len(good)
for match in sorted(good, key=lambda val: val.distance):
    print match.distance

In [ ]:
def drawMatches(img1, kp1, img2, kp2, matches, masks):
    """
    My own implementation of cv2.drawMatches as OpenCV 2.4.9
    does not have this function available but it's supported in
    OpenCV 3.0.0

    This function takes in two images with their associated 
    keypoints, as well as a list of DMatch data structure (matches) 
    that contains which keypoints matched in which images.

    An image will be produced where a montage is shown with
    the first image followed by the second image beside it.

    Keypoints are delineated with circles, while lines are connected
    between matching keypoints.

    img1,img2 - Grayscale images
    kp1,kp2 - Detected list of keypoints through any of the OpenCV keypoint 
              detection algorithms
    matches - A list of matches of corresponding keypoints through any
              OpenCV keypoint matching algorithm
    masks - a list of which matches are inliers (1?) or outliers (0?)
    """

    # Create a new output image that concatenates the two images together
    # (a.k.a) a montage
    rows1 = img1.shape[0]
    cols1 = img1.shape[1]
    rows2 = img2.shape[0]
    cols2 = img2.shape[1]

    out = np.zeros((max([rows1,rows2]),cols1+cols2,3), dtype='uint8')

    # Place the first image to the left
    out[:rows1,:cols1] = np.dstack([img1, img1, img1])

    # Place the next image to the right of it
    out[:rows2,cols1:] = np.dstack([img2, img2, img2])

    # For each pair of points we have between both images
    # draw circles, then connect a line between them
    for mask, mat in zip(masks,matches):

        # Get the matching keypoints for each of the images
        img1_idx = mat.queryIdx
        img2_idx = mat.trainIdx

        # x - columns
        # y - rows
        (x1,y1) = kp1[img1_idx].pt
        (x2,y2) = kp2[img2_idx].pt

        # Draw a small circle at both co-ordinates
        # radius 4
        # colour blue
        # thickness = 1
        
        cv2.circle(out, (int(x1),int(y1)), 4, (255*mask, 0, 255*(1-mask)), 1)   
        cv2.circle(out, (int(x2)+cols1,int(y2)), 4, (255*mask, 0, 255*(1-mask)), 1)

        # Draw a line in between the two points
        # thickness = 1
        # colour blue
        cv2.line(out, (int(x1),int(y1)), (int(x2)+cols1,int(y2)), (255, 0, 0), 1)


    # Show the image
    cv2.imshow('Matched Features', out)
    cv2.waitKey(0)
    cv2.destroyWindow('Matched Features')

    # Also return the image if you'd like a copy
    return out

In [ ]:
# Show only the top 10 matches - also save a copy for use later
# out = drawMatches(img1, kp1, img2, kp2, good[:15], mask)

In [ ]:
src_pts = np.float32([kp1[match.queryIdx].pt for match in good]).reshape(-1,1,2)
dst_pts = np.float32([kp2[match.trainIdx].pt for match in good]).reshape(-1,1,2)
M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
matchesMask = mask.ravel().tolist()

print M

print mask

h,w = img1.shape
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
dst = cv2.perspectiveTransform(pts, M)



In [ ]:
img1.shape

In [ ]:
kai = np.asarray(img1_3channel, dtype='float32')
print kai.shape
print M.shape

In [ ]:
img_T = cv2.warpPerspective(img1, M, img2.shape[::-1])

In [ ]:
plt.imshow(img2)

In [ ]:
plt.imshow(img1)

In [ ]:
from scipy.linalg import norm
from scipy import sum, average

def compare_images(img1, img2):
    # normalize to compensate for exposure difference, this may be unnecessary
    # consider disabling it
    img1 = normalize(img1)
    img2 = normalize(img2)
    # calculate the difference and its norms
    diff = img1 - img2  # elementwise for scipy arrays
    m_norm = sum(abs(diff))  # Manhattan norm
    z_norm = norm(diff.ravel(), 0)  # Zero norm
    return (m_norm, z_norm)

def to_grayscale(arr):
    "If arr is a color image (3D array), convert it to grayscale (2D array)."
    if len(arr.shape) == 3:
        return average(arr, -1)  # average over the last axis (color channels)
    else:
        return arr

def normalize(arr):
    rng = arr.max()-arr.min()
    amin = arr.min()
    return (arr-amin)*255/rng

In [ ]:
# compare_images(img_T, img2)

In [ ]:
compare_images(img_T, img2)

In [ ]:
print np.sum(img_T - img2)
print sum(sum(img_T - img2))

In [ ]:
print np.sum(img_T - img2)
print sum(sum(img_T - img2))

In [ ]:
plt.imshow(img_T)

In [ ]:
out = drawMatches(img1, kp1, img2, kp2, good, matchesMask)

In [ ]:
a = np.array([[1, 2], [4, 5], [7, 8]], dtype='float32')
h = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype='float32')
a = np.array([a])

pointsOut = cv2.perspectiveTransform(kai[:,:,:2], M)
print a
# print h
print pointsOut

In [ ]:
plt.imshow(pointsOut)